In [ ]:
# https://youtu.be/QogFIWa1YMg?list=PLA0M1Bcd0w8ynD1umfubKq1OBYRXhXkmH
%env TF_CPP_MIN_LOG_LEVEL=2

import tensorflow as tf  # noqa: E402
from tensorflow.keras.datasets import mnist  # noqa: E402
from tensorflow.keras.utils import to_categorical  # noqa: E402
import matplotlib.pyplot as plt
%matplotlib inline


class DenseLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units  # number of neurons

    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b",
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
        )

    def call(self, inputs):
        # regular = 100.0 * tf.reduce_mean(tf.square(self.w))
        # self.add_loss(regular)
        return tf.matmul(inputs, self.w) + self.b


In [ ]:
class NeuralNetwork(tf.keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.layer_1 = DenseLayer(128)
        self.layer_2 = DenseLayer(num_classes)

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.layer_2(x)
        x = tf.nn.softmax(x)
        return x

In [ ]:
model = NeuralNetwork()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    # optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),  # for M1/M2 Mac
    loss=tf.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

# model.compile(
#     optimizer="adam",
#     # optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),  # for M1/M2 Mac
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )

In [ ]:
# Load MNIST dataset
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Preprocess data
x_train = tf.cast(X_train.reshape((-1, 28 * 28)) / 255.0, tf.float32)
x_test = tf.cast(X_test.reshape((-1, 28 * 28)) / 255.0, tf.float32)

y_train = to_categorical(Y_train, 10)
y_test_cat = to_categorical(Y_test, 10)

In [ ]:
# Learning...
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    # validation_data=(x_test, y_test_cat),
    # verbose=2,
)

In [ ]:
# Testing...
model.evaluate(x_test, y_test_cat)

In [ ]:
tf.random.uniform((1, 10), minval=0, maxval=1, dtype=tf.int32)